# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [11]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import nltk
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [12]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

---
## Лабораторная работа 6.1

### Форматирование строк

6\.1\.1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [13]:
import re
import pandas as pd
from bs4 import BeautifulSoup
# Task 6.1.1
print("Задание 6.1.1")

recipes = pd.read_csv("../data/recipes_sample.csv")

id = recipes.id.sample(n=5).to_list()
minutes = recipes.minutes.sample(n=5).to_list()

wide = 0
for i in range(max(len(id), len(minutes))):
    t = max(len(str(id[i])), len(str(minutes[i])))
    if wide < t:
        wide = t
wide += 3

print(f"|{'id':^{wide}}|{'minutes':^{wide}}|")
print("|" + "-"*(wide*2 + 1) + "|")
for i in range(5):
    print(f"|{id[i]: ^{wide}}|{minutes[i]: ^{wide}}|")

Задание 6.1.1
|   id    | minutes |
|-------------------|
| 121014  |   45    |
| 265305  |   70    |
| 458158  |   15    |
| 175109  |   17    |
| 257638  |   20    |


6\.1\.2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [14]:
# Task 6.1.2
print("Задание 6.1.2")


def show_info(recipe_id=-1, is_step=False):
    result_str = ''
    recipes = pd.read_csv('../data/recipes_sample.csv')
    data = recipes[recipes['id'] == recipe_id]

    if recipe_id != -1:
        name = data['name'].item()
        with open('../data/steps_sample.xml', 'r') as fp:
            steps = BeautifulSoup(fp, 'xml')
            for recipe in steps.find_all('recipe'):
                if recipe.find("id").text == str(recipe_id):
                    steps_str = [step.text for step in recipe.find_all('step')]
                    break

    result_str += '"' + name.title() + '"\n\n'

    for index in range(len(steps_str)):
        result_str += str(index + 1) + '. ' + steps_str[index].capitalize() + '\n'

    result_str += 10 * "-" + "\nАвтор: " + str(data['contributor_id'].item()) + '\nСреднее время приготовления: ' + str(
        data["minutes"].item()) + " минут"

    if is_step:
        return steps_str
    else:
        return result_str
print(show_info(170895))

Задание 6.1.2
"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , pr

In [15]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

TypeError: show_info() got an unexpected keyword argument 'name'

## Работа с регулярными выражениями

6\.1\.3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [ ]:
# Task 6.1.3
print("задание 6.1.3")
recipes = pd.read_csv('../data/recipes_sample.csv')
data = show_info(25082, is_step=True)
pattern = re.compile(r"([\d]+ hour[s]*|[\d]+ minute[s]*)")
print(pattern.findall('\n'.join(data)))

6\.1\.4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [ ]:
# Task 6.1.4
print("Задание 6.1.4")
pattern = re.compile(r"^this.*[,] ?but")
data = pd.Series(recipes.description).dropna()
after_pattern = data.str.match(pattern)
print(after_pattern.sum())
print(data[after_pattern].sample(n=3))

## Лабораторная работа 6.2

6\.2\.1\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [16]:
# Task 6.2.1
print("Задание 6.2.1")
with open("../data/steps_sample.xml") as file:
    recipes_xml = BeautifulSoup(file, "xml")
    lst = []
    pattern = re.compile(r"[0-9] / [0-9]")
    for i in recipes_xml.recipes.find_all("recipe"):
        if i.id.contents[0] == "72367":
            steps = i.find_all("step")
            for j in steps:
                lst.append(" ".join(j.contents))
            print("before:", lst)
            for z in range(len(lst)):
                t = re.findall(pattern, lst[z])
                if t:
                    t_without_spaces = t[0].replace(' ', '')
                    lst[z] = lst[z].replace(t[0], t_without_spaces)
            print("after:", lst)

Задание 6.2.1
before: ['mix butter , flour , 1 / 3 c', 'sugar and 1-1 / 4 t', 'vanilla', 'press into greased 9" springform pan', 'mix cream cheese , 1 / 4 c', 'sugar , eggs and 1 / 2 t', 'vanilla beating until fluffy', 'pour over dough', 'combine apples , 1 / 3 c', 'sugar and cinnamon', 'arrange on top of cream cheese mixture and sprinkle with almonds', 'bake at 350 for 45-55 minutes , or until tester comes out clean']
after: ['mix butter , flour , 1/3 c', 'sugar and 1-1/4 t', 'vanilla', 'press into greased 9" springform pan', 'mix cream cheese , 1/4 c', 'sugar , eggs and 1/2 t', 'vanilla beating until fluffy', 'pour over dough', 'combine apples , 1/3 c', 'sugar and cinnamon', 'arrange on top of cream cheese mixture and sprinkle with almonds', 'bake at 350 for 45-55 minutes , or until tester comes out clean']


### Сегментация текста

6\.2\.2\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [17]:
# Task 6.2.2
print("Задание 6.2.2")
with open('../data/steps_sample.xml') as file:
    recipes_xml = BeautifulSoup(file, 'xml')
    set_w = set()
    for i in recipes_xml.find_all('recipe'):
        set_w.update(nltk.word_tokenize(i.steps.text))
    set_wo = set_w.copy()
    for i in set_wo:
        if i.isalpha() == False:
            set_w.remove(i)
    print(len(set_w))

Задание 6.2.2
14922


6\.2\.3\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [18]:
# Task 6.2.3
print("Задание 6.2.3")
with open('../data/steps_sample.xml') as file:
    recipes = pd.read_csv('../data/recipes_sample.csv')
    recipes_xml = BeautifulSoup(file, 'xml')
    recipes_d = recipes['description'].fillna("null").apply(nltk.sent_tokenize)
    [print("Length:", len(i), i) for i in sorted(recipes_d, key=lambda recipes_d: len(recipes_d), reverse=True)[:5]]


Задание 6.2.3
Length: 76 ['this wonderful icing is used for icing cakes and cookies as well as for borders and art work on cakes.', 'it makes a delicious filling also between the layers of cakes and under fondant icing.', 'you can make roses but it takes 3 or more days to dry them depending on the humidity.', 'there are many versions of “buttercream” icing.', 'some are made with eggs and all butter.', 'some varieties, you have to cook your sugar to a softball stage.', 'others are 100% shortening or a combination of shortening and butter.', 'each decorator has his or her favorite.', 'i personally think that the best taste and textured recipe is the one that has you cook your sugar, add to whipped eggs and use pounds of butter per batch.', 'but….', 'i live in a state that can easily be a 100 degrees for days on end during the summer and you know what butter does on hot days.', 'it melts!', 'a greasy puddle of melted icing on a cake plate is not something i want to look at or eat.', 'your

6\.2\.4\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.

In [19]:
# Task 6.2.4
print("Задание 6.2.4")


def func_6_2_4(number_of_sentence):
    recipes = pd.read_csv('../data/recipes_sample.csv')
    inform = nltk.pos_tag(nltk.word_tokenize(recipes[recipes['id'] == number_of_sentence].name.values[0]))
    print(''.join([f"{j:{len(i) + 1}}" for i, j in inform]) + '\n' + ''.join([f"{i:{len(i) + 1}}" for i, j in inform]))


func_6_2_4(241106)

Задание 6.2.4
JJ       NNS    IN   NNS       VBP  JJ     CC  JJ    NNS    
eggplant steaks with chickpeas feta cheese and black olives 
